In [33]:

import numpy as np
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import ipywidgets as widgets
from IPython.html.widgets import *
import pickle
import h5py 
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.models import Sequential
from keras.layers import TimeDistributed
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Activation, Dropout, Flatten, Dense, Lambda
from keras.layers import *
from keras.optimizers import Adam
from keras.models import *


In [2]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [3]:
import pandas as pd

In [4]:
def constant_illumination(img): 
    hsv_img = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    hsv_img[:,:,2] = hsv_img[:,:,2] * 0.8
    image_rgb = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2RGB)
    return image_rgb

In [5]:
def region_of_interest(img):
  h,w,c = img.shape
  h1= h-120
  crop_img = img[35:h1,:]
  image = cv2.resize(crop_img, (220, 66), interpolation = cv2.INTER_AREA)
  return image



In [6]:
def opticalFlowDense(image_current, image_next):
    """
    input: image_current, image_next (RGB images)
    calculates optical flow magnitude and angle and places it into HSV image
    * Set the saturation to the saturation value of image_next
    * Set the hue to the angles returned from computing the flow params
    * set the value to the magnitude returned from computing the flow params
    * Convert from HSV to RGB and return RGB image with same size as original image
    """

    gray_current = cv2.cvtColor(image_current, cv2.COLOR_RGB2GRAY)
    gray_next = cv2.cvtColor(image_next, cv2.COLOR_RGB2GRAY)

    hsv = np.zeros((image_current.shape))
    # set saturation
    hsv[:, :, 1] = cv2.cvtColor(image_next, cv2.COLOR_RGB2HSV)[:, :, 1]

    # Flow Parameters
    #     flow_mat = cv2.CV_32FC2
    flow_mat = None
    image_scale = 0.5
    nb_images = 1
    win_size = 15
    nb_iterations = 2
    deg_expansion = 5
    STD = 1.3
    extra = 0
    # obtain dense optical flow paramters
    flow = cv2.calcOpticalFlowFarneback(gray_current, gray_next,
                                        flow_mat,
                                        image_scale,
                                        nb_images,
                                        win_size,
                                        nb_iterations,
                                        deg_expansion,
                                        STD,
                                        0)

    # convert from cartesian to polar
    mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])

    # hue corresponds to direction
    hsv[:, :, 0] = ang * (180 / np.pi / 2)

    # value corresponds to magnitude
    hsv[:, :, 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)

    # convert HSV to int32's
    hsv = np.asarray(hsv, dtype=np.float32)
    rgb_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)

    return rgb_flow

In [7]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [8]:
speeds = pd.read_csv('/content/gdrive/My Drive/speeds2.csv')

In [9]:
speeds.info

<bound method DataFrame.info of           Speeds
0      28.105569
1      28.105569
2      28.106527
3      28.130404
4      28.109243
...          ...
20395   2.329180
20396   2.289795
20397   2.292917
20398   2.260600
20399   2.206759

[20400 rows x 1 columns]>

In [10]:
path1 = '/content/gdrive/My Drive/train.mp4'

In [11]:
cap = cv2.VideoCapture(path1)

In [12]:
no_of_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [13]:
X = []
y=[]

In [14]:
c = 0
o_frame = 0
o_speed = 0
while(c<no_of_frames):
  ret,n_frame = cap.read()
  n_speed = speeds['Speeds'][c]
  if c==0:
    o_frame = n_frame
    o_speed = n_speed
    c = c+1
    continue
  
  else:
    o_frame1 = constant_illumination(o_frame)
    o_frame1 = region_of_interest(o_frame1)

    n_frame1 = constant_illumination(n_frame)
    n_frame1 = region_of_interest(n_frame1)
    
    optical_flow = opticalFlowDense(o_frame1,n_frame1)
    avg_speed = np.mean([o_speed,n_speed])

  X.append(optical_flow)
  y.append(avg_speed)
  o_frame = n_frame
  o_speed = n_speed
  c = c+1




In [15]:
len(X)

20399

In [38]:
def cnn_lstm_model():
    ip_size = Input((66,220,3))
    # inputs = Input(ip_size)

    # model = Sequential()
    # model.add(Lambda(lambda x: x/ 127.5 - 1, input_shape = inputShape))

    conv1 = Convolution2D(10, (5, 5),kernel_initializer = 'he_normal',padding='same',
                            activation = 'elu')(ip_size)
    # model.add(conv1)
    # conv1 = TimeDistributed(conv1)

                            
    
    
    conv2 = Convolution2D(20, (5, 5),  
                            kernel_initializer = 'he_normal',padding='same',
                            activation = 'elu')(conv1)
    # model.add(conv2)
    # conv2 = TimeDistributed(conv2)


    
    conv3 = Convolution2D(40, (5, 5), 
                            kernel_initializer = 'he_normal',padding='same',
                            activation = 'elu')(conv2)
    # model.add(conv3)
    # conv3 = TimeDistributed(conv3)


    
    conv4 = Convolution2D(50, (3, 3), 
                            kernel_initializer = 'he_normal',padding='same',
                            activation = 'elu')(conv3)
    # model.add(conv4)
    # conv4 = TimeDistributed(conv4)


    
                
    conv5 = Convolution2D(60, (3, 3), 
                            kernel_initializer = 'he_normal',padding='same',
                            activation = 'elu')(conv4)
    # model.add(conv5)
    # conv5 = TimeDistributed(conv5)



    conv6 = Convolution2D(10, (3, 3), 
                            kernel_initializer = 'he_normal',padding='same',
                            activation = 'elu')(conv5)
    # model.add(conv6)
    # conv6 = TimeDistributed(conv6)

    # model.add(Flatten())

    # flatten = TimeDistributed(flatten)
    flatten = Flatten()(conv6)
    # layer = np.array(flatten)
    layer = Reshape((1,145200))(flatten)



    # model.add(LSTM((200),input_shape=(1,1,145200),return_sequences=True))
    lstm = LSTM((200),input_shape=(1,1,145200),return_sequences=False)(layer)
    dense1 = Dense(50,activation='relu',kernel_initializer='he_normal')(lstm)
    # model.add(Dense(50, activation='relu',kernel_initializer='he_normal'))
    dense2 = Dense(1)(dense1)
    # model.add(Dropout(0.5))
    # model.add(Dense(1))
    
    # op = Dense(1)(dense)
    adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model = Model(input = ip_size,output=dense2)
    model.compile(optimizer = adam, loss = 'mse')

    return model

In [19]:
callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=1, min_delta = 0.23,mode='min',),
             ModelCheckpoint('model_speed_pred.h5', monitor = 'val_loss',save_best_only = True, mode = 'min', verbose = 1,save_weights_only = True)]

In [20]:
X_train = X[:14279]
y_train = y[:14279]

X_test = X[14279:]
y_test = y[14279:]

In [21]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [43]:
y=np.array(y)

In [44]:
y.shape

(20399,)

In [41]:
y_train.shape

(14279,)

In [45]:
X_train.shape

(14279, 66, 220, 3)

In [42]:
y_train[:10]

array([28.105569 , 28.106048 , 28.1184655, 28.1198235, 28.0989075,
       28.0613915, 28.026351 , 28.0025575, 28.001488 , 27.999169 ])

In [46]:
y_train = np.reshape(y_train, (y_train.shape[0],1,1))
y_test = np.reshape(y_test, (y_test.shape[0],1,1))

In [35]:
model = cnn_lstm_model()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:74: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [49]:
model.fit(X_train,y_train,verbose=1,epochs=50,batch_size=32,callbacks = callbacks)

Epoch 1/50
14279/14279 [==============================] - 162s 11ms/step - loss: 58.6220
Epoch 2/50


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


14279/14279 [==============================] - 163s 11ms/step - loss: 58.6178
Epoch 3/50
14279/14279 [==============================] - 162s 11ms/step - loss: 58.5997
Epoch 4/50
14279/14279 [==============================] - 163s 11ms/step - loss: 58.5820
Epoch 5/50
14279/14279 [==============================] - 162s 11ms/step - loss: 58.5650
Epoch 6/50
14279/14279 [==============================] - 163s 11ms/step - loss: 58.5561
Epoch 7/50
14279/14279 [==============================] - 162s 11ms/step - loss: 58.5521
Epoch 8/50
14279/14279 [==============================] - 161s 11ms/step - loss: 58.5541
Epoch 9/50
14279/14279 [==============================] - 161s 11ms/step - loss: 58.5384
Epoch 10/50
14279/14279 [==============================] - 160s 11ms/step - loss: 58.5225
Epoch 11/50
14279/14279 [==============================] - 160s 11ms/step - loss: 58.5167
Epoch 12/50
14279/14279 [==============================] - 160s 11ms/step - loss: 58.5015
Epoch 13/50
14279/14279 [====

In [50]:
score = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:',score )
# print('Test accuracy:', score[1])


6120/6120 [==============================] - 16s 3ms/step
Test loss: 110.59119597618876
